In [1]:
from z3 import *
from spacer_tutorial import *

In [105]:
T = Solver()
x, x_out = Ints("x0 x1")

Init = x == 0
Tr = Or(And(x < 5, x_out == x + 1), And(x >= 5, x_out == x))
Bad = And(x >= 6)

In [ ]:
T = Solver()
F2 =[Not(And(x >= 6))]
T.add(*F2)
T.add(Bad)
T.check()
T.model()

In [108]:
#pob = x_out == T.model().eval(x)
pob = Bad
pob

And(x0 >= 6)

In [112]:
S = Solver()
F1 = [Not(x >= 6)]
S.add(*F1)
S.add(Tr)
S.add(pob)
res = S.check()
if res == sat:
    display(S.model())
else:
    display(res)


unsat

In [110]:

pob_1 = And(x_out >= 6)

In [111]:
S1 = Solver()
F0 = [Init]
S1.add(*F0)
S1.add(Tr)
S1.add(pob_1)
res = S1.check()
if res == sat:
    display(S.model())
else:
    display(res)

unsat